# Goals

Implement the simple generating process given in "Algorithm 2" in our simpliciality file on Overleaf, with weighting scheme given by Example 5.1 and $\Theta_{k}$ being exactly the set of all previously-seen hyperedges (optionally allowing "the full graph" with some multiplicity)

In [1]:
import numpy as np

In [2]:
# Create a graph according to Algorithm 2 in our Overleaf.

# V: integer - number of vertices.
# n: nd array of integers of length K_max, with n[k] being the number of hyperedges of size k+2. 
# p: ndarray of height V and width K_max, with p[v,k] being the weight of vertex v for hypergraphs of size k. If an array of length V is passed, it should be copied K_max times to make an array.
# beta: ndarray of height V and width K_max, with beta[v,k] being the weighting scheme seen in Example 5.1. If an array of length V or K_max is passed, it should be copied to make the 2d array. If a number is passed, it should be copied to make the array.
# Global (default = None): if not None, should be a list of integers of length K_max, with Global[k] saying how often to add the set "all vertices" to $\Theta_{k}$. If an integer is passed, it should be copied K_max times to make a list.

# NOTE: I make a token effort to make this modular by allowing one to sub in other strategies for e.g. "simple_upweight" (which would require replacing "get_alpha" with a generic "get_parameters" and replacing all of the preprocessing stuff). 

def Soft_CL(V,n,p,beta,Global=None,eps = (0.1)**10):
    # Preprocessing to make sure everything is the right shape.
    K_max = n.shape[0]
    p = Proc_p(p,K_max)
    beta = Proc_beta(beta,V,K_max)

    # Initialize
    H_list = [] # Elements will be the hyperedges (set objects)
    
    for k in range(K_max-1,-1,-1): # loop over size of hyperedge, from biggest to smallest
        # Freeze the hyperedge list, as the algorithm does
        Theta = H_list.copy()
        if k == (K_max-1): # In this case the list is empty, so add the "global" background. Note I'm wasting computational time doing this, since I don't need to compute alpha here. Oh well.
            Theta.append(np.array([x for x in range(V)]))
        if Global is not None:
            for i in range(Global[k]): # OK, this is surely a silly way to do this. Oh well.
                Theta.append(np.array([x for x in range(V)]))
        alpha = get_alpha(Theta,beta,V,k,eps=eps) # Compute alpha.
        for ind in range(n[k]): # make n[k] hyperedges
            adj_p = simple_upweight(Theta,p,V,alpha,beta,k) # Choose an element of theta, adjust p.
            H_list.append(make_CL_edge(k+2,adj_p))
    return(H_list)
    

In [3]:
# Makes a single hyperedge of size k according to soft Chung-Lu with weights p.
# CHECKED BASIC FUNCTION
def make_CL_edge(k,p):
    return(np.random.choice(V,size=k,p=p))

In [4]:
# Computes alpha, as in Example 5.1 of Overleaf. Note that at this point k is "fixed" and so is not included as an index where it is not needed.
# CHECKED BASIC FUNCTION
def get_alpha(Theta,beta,V,k,eps = (0.1)**10):
    # Compute gamma
    gamma = np.zeros(V)
    for edge in Theta:
        for v in np.unique(edge): # Computing probabilities, so we don't want to deal with self-loops.
            gamma[v] = gamma[v] + 1 # Doing raw counts.
    for v in range(V):
        gamma[v] = gamma[v]/len(Theta) # normalize
        
    # Compute alpha
    alpha = np.zeros(V)
    for v in range(V):
        alpha[v] = (1 - beta[v,k]*(1-gamma[v]))/(eps + gamma[v]) # Include small "eps" value to reduce numerical issues at cost of small bias.
    return(alpha)

In [5]:
# Does the simple upweight 
# CHECKED BASIC FUNCTION
def simple_upweight(Theta,p,V,alpha,beta,k):
    m = len(Theta)

    S = Theta[np.random.randint(m)] # Choose a random hyperedge to "be inside"
    adj_p = np.zeros(V)
    for v in range(V):
        if v in S:
            adj_p[v] = p[v,k]*alpha[v]
        else:
            adj_p[v] = p[v,k]*beta[v,k]
    tot = sum(adj_p)
    for v in range(V):
        adj_p[v] = adj_p[v]/tot
    return(adj_p)

In [6]:
# Preprocesses p into a matrix of the appropriate size/shape.
# CHECKED BASIC FUNCTION
def Proc_p(p,K_max):
    if len(p.shape) == 2:
        return(p)
    if len(p.shape) == 1:
        V = p.shape[0]
        p = np.tile(p, K_max).reshape(K_max,V).transpose()
        return(p)
    return(-1) # error
        

In [7]:
# Preprocesses beta into a matrix of the appropriate size/shape.
# CHECKED BASIC FUNCTION
def Proc_beta(beta,V,K_max):
    res = np.zeros((V,K_max))
    if type(beta) is float:
        res.fill(beta)
        return(res)
    if len(beta.shape) == 2:
        return(beta)
    if len(beta.shape) == 1:
        if beta.shape[0] == V:
            beta = np.tile(beta, K_max).reshape(K_max,V).transpose()
            return(beta)
        if beta.shape[0] == K_max:
            beta = np.tile(beta, V).reshape(V,K_max)
            return(beta)
    return(-1) # error

## Sanity Check - does something

Do the above main function and helper functions do something plausible?

In [9]:

# Setting up choices
# K_max
K_max = 4
k=3

# p
p = np.array([0.3,2,0.45,3,8,0.22,0.5,1.2,2.3,2.5])
p = p/sum(p)
p = Proc_p(p,K_max)

#V
V = p.shape[0]

# Theta
Theta = []
Theta.append([5,6,7,8])
Theta.append([1,6,3,8])
Theta.append([2,6,4,3])
Theta.append([0,1,2,3,4,5,6,7,8,9])

# beta
beta = Proc_beta(0.2,V,K_max)

# alpha
alpha = get_alpha(Theta,beta,V,k)

# n
n = np.array([0,3,3,1])

Soft_CL(V,n,p,beta,Global=None,eps = (0.1)**10)

[array([7, 4, 1, 8, 4]),
 array([1, 4, 4, 4]),
 array([4, 4, 1, 9]),
 array([8, 4, 7, 4]),
 array([4, 4, 8]),
 array([7, 8, 7]),
 array([4, 4, 4])]

## Sanity check - not "too slow"

Let's see how long this takes to make a graph with a a few hundred hyperedges.



In [10]:
%%time
V = 1000
n=np.array([222,84,33,12,4,1])
p = np.asarray([np.random.randint(15,30) for x in range(V)])
tot = sum(p)
for v in range(V):
    p[v] = p[v]/tot
beta = 0.2



CPU times: user 2.2 ms, sys: 0 ns, total: 2.2 ms
Wall time: 2.2 ms


In [11]:
%%time
res = Soft_CL(V,n,p,beta,Global=None,eps = (0.1)**10)

CPU times: user 999 ms, sys: 0 ns, total: 999 ms
Wall time: 999 ms


In [12]:
len(res)

356

# Allowing for bias at the start

Rewrite the above, allowing for a user-specified "bias" function.

In [13]:
import random

random.choices([1,2,3,4,5],k=3)

[1, 4, 2]

In [17]:
# Rewriting Soft_CL with an extra optional parameter:

# init_noise: a callable random number generator
# n_init_noise: how many vertices to apply the above to
# p_init_noise: parameters for the above
def Soft_CL_V2(V,n,p,beta,init_noise=None, n_init_noise = None, p_init_noise = None,Global=None,eps = (0.1)**10):
    # Preprocessing to make sure everything is the right shape.
    K_max = n.shape[0]
    p = Proc_p(p,K_max)
    beta = Proc_beta(beta,V,K_max)

    if init_noise is not None:
        noisy_p = p
        adjusted_vertices = random.choices([x for x in range(V)])
        for v in adjusted_vertices:
            noisy_p[v,K_max - 1] = init_noise(noisy_p[v,K_max - 1], p_init_noise)
        tot = sum(noisy_p[:,K_max - 1])
        for v in range(V):
            noisy_p[v,K_max - 1] = noisy_p[v,K_max - 1]/tot
        p = noisy_p
        

    # Initialize
    H_list = [] # Elements will be the hyperedges (set objects)
    
    for k in range(K_max-1,-1,-1): # loop over size of hyperedge, from biggest to smallest
        # Freeze the hyperedge list, as the algorithm does
        Theta = H_list.copy()
        if k == (K_max-1): # In this case the list is empty, so add the "global" background. Note I'm wasting computational time doing this, since I don't need to compute alpha here. Oh well.
            Theta.append(np.array([x for x in range(V)]))
        if Global is not None:
            for i in range(Global[k]): # OK, this is surely a silly way to do this. Oh well.
                Theta.append(np.array([x for x in range(V)]))
        alpha = get_alpha(Theta,beta,V,k,eps=eps) # Compute alpha.
        for ind in range(n[k]): # make n[k] hyperedges
            adj_p = simple_upweight(Theta,p,V,alpha,beta,k) # Choose an element of theta, adjust p.
            H_list.append(make_CL_edge(k+2,adj_p))
    return(H_list)
    

In [18]:
# A simple "noise function"

def exp_noise(x,scale):
    return(np.random.exponential(x*scale))

exp_noise(2,3)

1.9396943425453173

In [19]:
%%time
res_V2 = Soft_CL_V2(V,n,p,beta,init_noise = exp_noise, n_init_noise = 12, p_init_noise=2,Global=None,eps = (0.1)**10)

CPU times: user 986 ms, sys: 3.63 ms, total: 989 ms
Wall time: 989 ms


# SCRATCH: Trying Jordan's Statistics Function

In [21]:
import HypergraphFunctions as hf

In [22]:

# Setting up choices
# K_max
K_max = 4

# p
m1 = 200
m2 = 200
p = np.array([0.3]*m1 + [1.1]*m2)
p = p/sum(p)
p = Proc_p(p,K_max)

#V
V = p.shape[0]

# beta
beta = Proc_beta(0.2,V,K_max)


# n
n = np.array([0,55,32,4])

X = Soft_CL(V,n,p,beta,Global=None,eps = (0.1)**10)

In [27]:
# The expected input for Jordan's SimplicialMatrix function looks like:

vertices = [x for x in range(V)]
edges = X

In [39]:
hf.simplicialMatrix(vertices,edges)

[[0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 11, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0]]

In [40]:
cl_vertices, cl_edges = ChungLu(vertices,edges)
hf.simplicialMatrix(cl_vertices,cl_edges)

[[0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0]]

# Trying to empirically fit real data...

I'm going to load a real hypergraph, then try to "match" it to my model as well as I can by tweaking the single parameter beta...

Note that there is no reason to expect this to "work well" with a single parameter. We can presumably do much better by allowing many parameters (and extending the model), but for now we're not doing that.

In [41]:
import ReadHypergraph as rh

In [71]:
# Read the graph, compute simpliciality matrix
h_vertices, h_edges = rh.readHG("hospital-lyon-simple.txt")
h_SM = hf.simplicialMatrix(h_vertices,h_edges)
h_SM

[[0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1938, 347, 20],
 [0, 0, 0, 0, 190, 12],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0]]

In [66]:
# Given output of rh.readHG, get parameters for SoftCL model
def extractParams(vertices,edges):

    # Get number of hyperedges of each size
    h_sorted_edges = hf.sortBySize(h_edges)
    h_deg_list = [[x,len(h_sorted_edges[x])] for x in h_sorted_edges.keys()]
    
    # Get degree of each vertex
    h_degrees = degrees(h_vertices,h_edges)
    h_degrees = [x for x in h_degrees.values()]

    # K_max
    K_max = h_deg_list[len(h_deg_list)-1][0]
    
    # p
    p = np.asarray([x/sum(h_degrees) for x in h_degrees])
    p = Proc_p(p,K_max)
    
    #V
    V = p.shape[0]
    
    # beta
    beta_0 = Proc_beta(0.2,V,K_max)
    
    # n
    n = [x[1] for x in h_deg_list]
    n = np.asarray(n)

    return(V,n,p,beta)

In [74]:
# Run it...
V,n,p,beta = extractParams(h_vertices,h_edges)

sample = Soft_CL(V,n,p,beta,Global=None,eps = (0.1)**10)
s_vertices = [x for x in range(V)]
s_edges = sample
s_mat = hf.simplicialMatrix(s_vertices,s_edges)

In [82]:
# Let's do stochastic gradient descent.

def OptimizeBeta(beta_0, V,n,p, comp_function, T=100, T_steps = None, eps = (0.1)**5):
    if T_steps is None:
        T_steps = [1/(x+1) for x in range(T)]
    vertices = [x for x in range(V)]
    beta = beta_0
    beta_list = [beta_0]
    for t in T_steps:
        sample = Soft_CL(V,n,p,beta,Global=None,eps = (0.1)**10)
        edges = sample
        h_SM = hf.simplicialMatrix(h_vertices,h_edges)
        beta = min(1-eps,max(eps,beta + t*comp_function(h_SM)))
        beta_list.append(beta)
    return(beta_list)

In [88]:
from functools import partial

def mat_comp(A,B):
    m_a = len(A)
    m_b = len(B)
    n_a = len(A[0])
    n_b = len(B[0])
    if m_a != m_b:
        return(0)
    if n_a != n_b:
        return(0)
    tot = 0
    for i in range(m_a):
        for j in range(n_a):
            tot += A[i][j] - B[i][j]
    if tot > 0:
        return -1
    return 1

hosp_comp = partial(mat_comp,h_SM)

In [89]:
%%time
# Trying it out
beta_0 = 0.2
foo = OptimizeBeta(beta_0, V,n,p, hosp_comp, T=100, T_steps = None, eps = (0.1)**5)

CPU times: user 43.5 s, sys: 58.5 ms, total: 43.6 s
Wall time: 43.6 s


In [92]:
# Not great
foo[99]

0.99999